In [2]:
# Example Notebook Workflow
from sentinel2py.downloader.manager import Sentinel2Manager
import pandas as pd

# Initialize manager
manager = Sentinel2Manager(out_dir="./data")

In [3]:
bbox = [7.0, 45.9, 8.0, 46.3]  # Example bounding box (lon/lat)
start_date = "2023-09-01"
end_date = "2023-09-30"
max_cloud_cover = 20
bands = ["B02", "B03", "B04", "B08"]  # Blue, Green, Red, NIR

In [4]:
items = manager.find_items(bbox, start_date, end_date, max_cloud_cover=max_cloud_cover)
print(f"Found {len(items)} tiles")

Found 28 tiles


In [5]:
metadata_list = []
for i, item in enumerate(items):
    metadata_list.append({
        "Index": i,
        "Tile_ID": item.properties.get("sentinel:tile_id", item.id),
        "Date": item.properties.get("datetime"),
        "Cloud_Cover_%": item.properties.get("eo:cloud_cover")
    })

df_metadata = pd.DataFrame(metadata_list)
display(df_metadata.sort_values(by="Cloud_Cover_%").head())  # Sort by cloud cover

,Index,Tile_ID,Date,Cloud_Cover_%
5,5,S2A_MSIL2A_20230926T102801_R108_T32TLR_2023092...,2023-09-26T10:28:01.024000Z,0.000332
4,4,S2A_MSIL2A_20230926T102801_R108_T32TLR_2024102...,2023-09-26T10:28:01.024000Z,0.000408
3,3,S2A_MSIL2A_20230926T102801_R108_T32TLS_2023092...,2023-09-26T10:28:01.024000Z,0.001672
2,2,S2A_MSIL2A_20230926T102801_R108_T32TMR_2023092...,2023-09-26T10:28:01.024000Z,0.005233
6,6,S2A_MSIL2A_20230926T102801_R108_T31TGM_2023092...,2023-09-26T10:28:01.024000Z,0.010714


In [6]:
# Select the least cloudy tile
tile_least_cloudy = manager.select_best(items, method="least_cloudy")
tile_least_cloudy

[SELECT] Least cloudy tile: S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025, Cloud: 0.000332%


<Item id=S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025>

In [7]:
# Or select tile by index
tile_index_0 = manager.select_best(items, method="index", index=0)

[SELECT] Tile by index 0: S2B_MSIL2A_20230928T101719_R065_T32TMS_20230928T172140


In [8]:
# Download multiple bands for a single tile
paths = manager.download_bands(tile_least_cloudy, bands, stack_strategy=None)
print("Downloaded bands (no stacking):", paths)

[DOWNLOAD] Downloading 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
[INFO] Starting download of 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
[INFO] Processing band: B02


[SUCCESS] Downloaded ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B02.tif
[INFO] Processing band: B03


[SUCCESS] Downloaded ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B03.tif
[INFO] Processing band: B04


[SUCCESS] Downloaded ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B04.tif
[INFO] Processing band: B08


[SUCCESS] Downloaded ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B08.tif
[INFO] Completed download of 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
Downloaded bands (no stacking): ['./data\\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\\B02.tif', './data\\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\\B03.tif', './data\\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\\B04.tif', './data\\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\\B08.tif']


In [9]:
# Download and stack bands using same resolution
stacked_same = manager.download_bands(tile_least_cloudy, bands, stack_strategy="same")
print("Stacked bands (same resolution):", stacked_same)

[DOWNLOAD] Downloading 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
[INFO] Starting download of 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
[INFO] Processing band: B02
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B02.tif already exists
[INFO] Processing band: B03
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B03.tif already exists
[INFO] Processing band: B04
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B04.tif already exists
[INFO] Processing band: B08
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B08.tif already exists
[INFO] Completed download of 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
[STACK] Using strategy: same
[STACK] Keeping original resolution: 10.0m
[STACK] Band added: B02.tif
[STACK] Band added: B03.tif
[STACK] Band added: B04.tif
[STACK] Band added: B08.tif
[STACK] Stacked file created: ./data\S2A

In [10]:
# Download and stack bands using highest resolution
stacked_highest = manager.download_bands(tile_least_cloudy, bands, stack_strategy="highest")
print("Stacked bands (highest resolution):", stacked_highest)

[DOWNLOAD] Downloading 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
[INFO] Starting download of 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
[INFO] Processing band: B02
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B02.tif already exists
[INFO] Processing band: B03
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B03.tif already exists
[INFO] Processing band: B04
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B04.tif already exists
[INFO] Processing band: B08
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B08.tif already exists
[INFO] Completed download of 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
[STACK] Using strategy: highest
[STACK] Resampling bands to 10.0m resolution
[STACK] Band added: B02.tif
[STACK] Band added: B03.tif
[STACK] Band added: B04.tif
[STACK] Band added: B08.tif
[STACK] Stacked file created: ./dat

In [11]:
# Download and stack bands to a user-defined resolution (e.g., 10m)
stacked_user_res = manager.download_bands(tile_least_cloudy, bands, stack_strategy=40)
print("Stacked bands (user-defined 10m):", stacked_user_res)

[DOWNLOAD] Downloading 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
[INFO] Starting download of 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
[INFO] Processing band: B02
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B02.tif already exists
[INFO] Processing band: B03
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B03.tif already exists
[INFO] Processing band: B04
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B04.tif already exists
[INFO] Processing band: B08
[SKIP] ./data\S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025\B08.tif already exists
[INFO] Completed download of 4 bands for tile S2A_MSIL2A_20230926T102801_R108_T32TLR_20230926T182025
[STACK] Using strategy: 40
[STACK] Resampling bands to 40m resolution
[STACK] Band added: B02.tif
[STACK] Band added: B03.tif
[STACK] Band added: B04.tif
[STACK] Band added: B08.tif
[STACK] Stacked file created: ./data\S2A_M